In [1]:
import org.apache.spark.SparkConf
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.functions._
import org.apache.spark.SparkConf
import org.apache.spark.ml.feature.{RegexTokenizer,StopWordsRemover,
                                    StringIndexer,CountVectorizer,
                                    CountVectorizerModel,VectorAssembler,
                                   IDF,OneHotEncoderEstimator}

import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit}

// import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics
// import org.apache.spark.ml.evaluation
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator

import org.apache.spark.ml.param.ParamMap


val conf = new SparkConf().setAll(Map(
      "spark.scheduler.mode" -> "FIFO",
      "spark.speculation" -> "false",
      "spark.reducer.maxSizeInFlight" -> "48m",
      "spark.serializer" -> "org.apache.spark.serializer.KryoSerializer",
      "spark.kryoserializer.buffer.max" -> "1g",
      "spark.shuffle.file.buffer" -> "32k",
      "spark.default.parallelism" -> "12",
      "spark.sql.shuffle.partitions" -> "12",
      "spark.driver.maxResultSize" -> "2g"
    ))

val spark = SparkSession
  .builder
  .config(conf)
  .appName("TP Spark : Trainer")
  .getOrCreate()


/*******************************************************************************
  *
  *       TP 3
  *
  *       - lire le fichier sauvegarder précédemment
  *       - construire les Stages du pipeline, puis les assembler
  *       - trouver les meilleurs hyperparamètres pour l'entraînement du pipeline avec une grid-search
  *       - Sauvegarder le pipeline entraîné
  *
  *       if problems with unimported modules => sbt plugins update
  *
  ********************************************************************************/

println("hello world ! from Trainer")

Intitializing Scala interpreter ...

Spark Web UI available at http://137.194.73.102:4040
SparkContext available as 'sc' (version = 2.4.4, master = local[*], app id = local-1573822561448)
SparkSession available as 'spark'


hello world ! from Trainer


import org.apache.spark.SparkConf
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.functions._
import org.apache.spark.SparkConf
import org.apache.spark.ml.feature.{RegexTokenizer, StopWordsRemover, StringIndexer, CountVectorizer, CountVectorizerModel, VectorAssembler, IDF, OneHotEncoderEstimator}
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit}
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.ml.param.ParamMap
conf: org.apache.spark.SparkConf = org.apache.spark.SparkConf@1106517
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@69007186


In [2]:
val df:DataFrame = spark.read.parquet("/home/jorge/Documents/Cours/Spark/RepoAdotTPs/data/prepared_trainingset/")

// df.select("project_id", "name", "desc", "goal").show(5)
// df.select("keywords", "final_status", "country2", "currency2").show(5)
// df.select("deadline2", "created_at2", "launched_at2", "days_campaign").show(5)
// df.select("hours_prepa", "text")

df: org.apache.spark.sql.DataFrame = [project_id: string, name: string ... 12 more fields]


In [3]:
val tokenizer = new RegexTokenizer()
  .setPattern("\\W+")
  .setGaps(true)
  .setInputCol("text")
  .setOutputCol("tokens")
// val dfTokenized = tokenizer.transform(df)


val remover = new StopWordsRemover()
  .setInputCol(tokenizer.getOutputCol)
  .setOutputCol("filtered")

// val dfsw = remover.transform(dfTokenized)
 

val cvModel: CountVectorizer = new CountVectorizer()
    .setInputCol(remover.getOutputCol)
    .setOutputCol("vect")
    .setMinDF(50)
    
    

// val dfv = cvModel.fit(dfsw).transform(dfsw)



val idf = new IDF()
    .setInputCol(cvModel.getOutputCol)
    .setOutputCol("tfidf")

// val idfModel = idf.fit(dfv)

// val rescaledData = idfModel.transform(dfv)
                                      

val indexerCountry = new StringIndexer()
  .setInputCol("country2")
  .setOutputCol("country_indexed")

val indexerCurrency = new StringIndexer()
  .setInputCol("currency2")
  .setOutputCol("currency_indexed")


// val indexedCountry = indexerCountry.fit(rescaledData).transform(rescaledData)
// val indexedCountryCurrency = indexerCurrency.fit(indexedCountry).transform(indexedCountry)


val encoder = new OneHotEncoderEstimator()
  .setInputCols(Array("country_indexed", "currency_indexed"))
  .setOutputCols(Array("country_onehot", "currency_onehot"))

// val model = encoder.fit(indexedCountryCurrency)
// val encoded = model.transform(indexedCountryCurrency)


val assembler = new VectorAssembler()
  .setInputCols(Array("tfidf","days_campaign","hours_prepa","goal","country_onehot","currency_onehot"))
  .setOutputCol("features")

val lr = new LogisticRegression()
  .setElasticNetParam(0.0)
  .setFitIntercept(true)
  .setFeaturesCol("features")
  .setLabelCol("final_status")
  .setStandardization(true)
  .setPredictionCol("predictions")
  .setRawPredictionCol("raw_predictions")
  .setThresholds(Array(0.7, 0.3))
  .setTol(1.0e-6)
  .setMaxIter(20)

// val transformed = assembler
//     .setHandleInvalid("skip")
//     .transform(encoded)
//     .drop("project_id","name","desc","goal","keywords",
//          "country2","currency2","deadline2","created_at2","launched_at2",
//          "days_campaign","hours_prepa","text","tokens","filtered","vect",
//          "country_indexed","currency_indexed","tfidf","days_campaign",
//           "hours_prepa","goal","country_onehot","currency_onehot")
    




tokenizer: org.apache.spark.ml.feature.RegexTokenizer = regexTok_10225ad89bea
remover: org.apache.spark.ml.feature.StopWordsRemover = stopWords_e62469a602ad
cvModel: org.apache.spark.ml.feature.CountVectorizer = cntVec_163225298fad
idf: org.apache.spark.ml.feature.IDF = idf_45fbefa93475
indexerCountry: org.apache.spark.ml.feature.StringIndexer = strIdx_1dac9ea4dfb8
indexerCurrency: org.apache.spark.ml.feature.StringIndexer = strIdx_0f9fb213412b
encoder: org.apache.spark.ml.feature.OneHotEncoderEstimator = oneHotEncoder_bb0c386815dd
assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_bc3574af7f54
lr: org.apache.spark.ml.classification.LogisticRegression = logreg_1c3d4522bf2f


In [16]:
// transformed.show(numRows=5,truncate=false)

val pipeline = new Pipeline()
  .setStages(Array(tokenizer, remover,cvModel,idf,indexerCountry,
                  indexerCurrency,encoder, assembler,lr ))

// Fit the pipeline to training documents.
val model = pipeline.fit(df)



pipeline: org.apache.spark.ml.Pipeline = pipeline_708c23f7cfe5
model: org.apache.spark.ml.PipelineModel = pipeline_708c23f7cfe5


In [21]:
model.transform(df).columns

// model.transform(df).select("text","predictions","final_status")
model.transform(df).groupBy("final_status", "predictions").count.show()

+------------+-----------+-----+
|final_status|predictions|count|
+------------+-----------+-----+
|           1|        0.0| 8415|
|           0|        1.0|26141|
|           1|        1.0|25978|
|           0|        0.0|47080|
+------------+-----------+-----+



In [9]:
val Array(train,test) = df.randomSplit(Array[Double](0.9, 0.1))
val size = (train.count,test.count)

train: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [project_id: string, name: string ... 12 more fields]
test: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [project_id: string, name: string ... 12 more fields]
size: (Long, Long) = (96968,10646)


In [11]:
val model2 = pipeline.fit(train)

model2: org.apache.spark.ml.PipelineModel = pipeline_a83e74e46e29


In [12]:
val predictions = model2.transform(test)

predictions.select("final_status","predictions","probability").show(100)

+------------+-----------+--------------------+
|final_status|predictions|         probability|
+------------+-----------+--------------------+
|           1|        0.0|[0.83346856290985...|
|           0|        0.0|[0.75592356170283...|
|           0|        0.0|[0.88582257302640...|
|           0|        1.0|[0.43505188248873...|
|           1|        0.0|[0.72957488453949...|
|           0|        0.0|[0.92849923107638...|
|           0|        0.0|[0.81460799809196...|
|           0|        1.0|[0.59479358906910...|
|           1|        0.0|[0.71505324314113...|
|           0|        0.0|[0.96793421536937...|
|           0|        1.0|[0.63416940279811...|
|           0|        1.0|[0.65282514269261...|
|           0|        1.0|[0.58268471533627...|
|           1|        0.0|[0.72290322572450...|
|           0|        1.0|[0.66616489673114...|
|           1|        1.0|[0.37431144811710...|
|           0|        1.0|[0.63116156355414...|
|           0|        0.0|[0.85599357146

predictions: org.apache.spark.sql.DataFrame = [project_id: string, name: string ... 24 more fields]


In [9]:
// // Clear the prediction threshold so the model will return probabilities
// model2.clearThreshold

// // Compute raw scores on the test set
// val predictionAndLabels = test.map { case LabeledPoint(label, features) =>
//   val prediction = model2.predict(features)
//   (prediction, label)
// }

// // Instantiate metrics object
// val metrics = new BinaryClassificationMetrics(predictionAndLabels)

// // Precision by threshold
// val precision = metrics.precisionByThreshold
// precision.foreach { case (t, p) =>
//   println(s"Threshold: $t, Precision: $p")
// }

// // Recall by threshold
// val recall = metrics.recallByThreshold
// recall.foreach { case (t, r) =>
//   println(s"Threshold: $t, Recall: $r")
// }

// // Precision-Recall Curve
// val PRC = metrics.pr

// // F-measure
// val f1Score = metrics.fMeasureByThreshold
// f1Score.foreach { case (t, f) =>
//   println(s"Threshold: $t, F-score: $f, Beta = 1")
// }

// val beta = 0.5
// val fScore = metrics.fMeasureByThreshold(beta)
// f1Score.foreach { case (t, f) =>
//   println(s"Threshold: $t, F-score: $f, Beta = 0.5")
// }

// // AUPRC
// val auPRC = metrics.areaUnderPR
// println(s"Area under precision-recall curve = $auPRC")

// // Compute thresholds used in ROC and PR curves
// val thresholds = precision.map(_._1)

// // ROC Curve
// val roc = metrics.roc

// // AUROC
// val auROC = metrics.areaUnderROC
// println(s"Area under ROC = $auROC")

In [13]:
val evaluator = new MulticlassClassificationEvaluator()
      .setLabelCol("final_status")
      .setPredictionCol("predictions")
      .setMetricName("f1")

val f1 = evaluator.evaluate(predictions)
println("Test set accuracy = " + f1)

Test set accuracy = 0.6560841473369508


evaluator: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_85715d799e77
f1: Double = 0.6560841473369508


In [14]:
import org.apache.spark.ml.tuning.{CrossValidator, ParamGridBuilder,CrossValidatorModel,TrainValidationSplit}
import org.apache.spark.ml.param.ParamMap



import org.apache.spark.ml.tuning.{CrossValidator, ParamGridBuilder, CrossValidatorModel, TrainValidationSplit}
import org.apache.spark.ml.param.ParamMap


In [15]:
val grid = new ParamGridBuilder()
    .addGrid(lr.regParam,Array(10e-8,10e-6,10e-4,10e-2))
    .build()

// val cv = new CrossValidator()
//   .setEstimator(pipeline)
//   .setEvaluator(evaluator)
//   .setEstimatorParamMaps(grid)
//   .setNumFolds(5)

// val cvModel = cv.fit(df)

val trainValidationSplit = new TrainValidationSplit()
  .setEstimator(pipeline)
  .setEvaluator(evaluator)
  .setEstimatorParamMaps(grid)
  // 80% of the data will be used for training and the remaining 20% for validation.
  .setTrainRatio(0.7)

val model = trainValidationSplit.fit(train)


grid: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	logreg_1c3d4522bf2f-regParam: 1.0E-7
}, {
	logreg_1c3d4522bf2f-regParam: 1.0E-5
}, {
	logreg_1c3d4522bf2f-regParam: 0.001
}, {
	logreg_1c3d4522bf2f-regParam: 0.1
})
trainValidationSplit: org.apache.spark.ml.tuning.TrainValidationSplit = tvs_5fc8a7d699d5
model: org.apache.spark.ml.tuning.TrainValidationSplitModel = tvs_5fc8a7d699d5


In [13]:
// model.bestModel.params.
model.bestModel.transform(test).select("name","final_status","predictions","probability").show(100)

+--------------------+------------+-----------+--------------------+
|                name|final_status|predictions|         probability|
+--------------------+------------+-----------+--------------------+
|northern lights o...|           0|        0.0|[0.78392009081449...|
|happy birthday, m...|           0|        0.0|[0.75705606894479...|
|           plexibots|           0|        0.0|[0.70923059165288...|
|           sellsafer|           0|        0.0|[0.95349249745577...|
|arresting power: ...|           1|        0.0|[0.77689872101372...|
|web page to revol...|           0|        0.0|[0.93434627040621...|
|        inchicago.co|           0|        1.0|[0.47398301919751...|
|project tangled -...|           0|        0.0|[0.73872767245204...|
|day of the dead b...|           1|        1.0|[0.24081599891754...|
|j.calarese&co mar...|           0|        0.0|[0.91734089576189...|
|official elfquest...|           1|        1.0|[0.31152821301541...|
|"kol presents fir...|           0

In [14]:
evaluator.evaluate(model.bestModel.transform(test))

res6: Double = 0.6468747625444753


In [103]:
val grid2 = new ParamGridBuilder()
    .addGrid(lr.regParam,Array(10e-8,10e-6,10e-4,10e-2))
    .addGrid(cvModel.minDF,Array(55.0,75.0,95.0))
    .build()


// val cv = new CrossValidator()
//   .setEstimator(pipeline)
//   .setEvaluator(evaluator)
//   .setEstimatorParamMaps(grid)
//   .setNumFolds(5)

// val cvModel = cv.fit(df)

val trainValidationSplit2 = new TrainValidationSplit()
  .setEstimator(pipeline)
  .setEvaluator(evaluator)
  .setEstimatorParamMaps(grid)
  // 80% of the data will be used for training and the remaining 20% for validation.
  .setTrainRatio(0.7)

val modelbis = trainValidationSplit2.fit(train)


grid2: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	cntVec_fd09521beb46-minDF: 55.0,
	logreg_7240eeb9b1b1-regParam: 1.0E-7
}, {
	cntVec_fd09521beb46-minDF: 75.0,
	logreg_7240eeb9b1b1-regParam: 1.0E-7
}, {
	cntVec_fd09521beb46-minDF: 95.0,
	logreg_7240eeb9b1b1-regParam: 1.0E-7
}, {
	cntVec_fd09521beb46-minDF: 55.0,
	logreg_7240eeb9b1b1-regParam: 1.0E-5
}, {
	cntVec_fd09521beb46-minDF: 75.0,
	logreg_7240eeb9b1b1-regParam: 1.0E-5
}, {
	cntVec_fd09521beb46-minDF: 95.0,
	logreg_7240eeb9b1b1-regParam: 1.0E-5
}, {
	cntVec_fd09521beb46-minDF: 55.0,
	logreg_7240eeb9b1b1-regParam: 0.001
}, {
	cntVec_fd09521beb46-minDF: 75.0,
	logreg_7240eeb9b1b1-regParam: 0.001
}, {
	cntVec_fd09521beb46-minDF: 95.0,
	logreg_7240eeb9b1b1-regParam: 0.001
}, {
	cntVec_fd09521beb46-minDF: 55.0,
	logreg_7240ee...

In [110]:
// modelbis.bestModel
evaluator.evaluate(modelbis.bestModel.transform(test))

res66: Double = 0.6500708947162648


In [111]:
evaluator.evaluate(model.bestModel.transform(test))

res67: Double = 0.6468747625444753
